In [ ]:
#@markdown Connect your google drive [Completed!!]


#Mounting and extracting all the PDFs
import zipfile
from google.colab import drive
import os
drive.mount('/content/drive/')
root_dir = '/content/drive/MyDrive/GenAI (Training-2)/Case Study_Batch 2-20230928T185455Z-001/Case Study_Batch 2/Data/PDF Documents'
print(root_dir)
#Extracting all PDF documents
#!unzip -u "/content/drive/My Drive/Gen AI Case Study/PDF Documents.zip" -d "/content/drive/My Drive/Gen AI Case Study/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/GenAI (Training-2)/Case Study_Batch 2-20230928T185455Z-001/Case Study_Batch 2/Data/PDF Documents


In [ ]:
#@markdown installing required liberaries [Completed!!]

!pip install redis langchain pypdf unstructured pdf2image pdfminer.six unstructured_inference pytesseract unstructured_pytesseract poppler-utils fitz pymupdf PyPDF2 tqdm seaborn spacy nltk
!pip install transformers==4.30 InstructorEmbedding sentence_transformers accelerate bitsandbytes xformers einops

!pip install sentence-transformers
!pip install json
!pip install -qU pinecone-client
!apt-get install poppler-utils
! apt install tesseract-ocr
! apt install libtesseract-dev
!pip install torchaudio
!pip install -Uq chromadb

In [ ]:

#@markdown Importing the relevant libraries  [Completed!!]
import time
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from langchain.document_loaders.pdf import PyPDFium2Loader, PyMuPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter,SpacyTextSplitter,NLTKTextSplitter
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib

from langchain.vectorstores import Chroma

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


In [ ]:
#@markdown Loading PDF dataset - (Set1, Set2) Using "PyPDFium2Loader" Parser [Completed!!]
#Loading all pdfs
#root_dir = "/content/drive/My Drive/Gen AI Case Study/"
pdf_folder_path_a = f'{root_dir}/Set1/'
pdf_folder_path_b = f'{root_dir}/Set2/'
print("Number of pdfs from set 1: " ,len(os.listdir(pdf_folder_path_a)))
print("Number of pdfs from set 2: " ,len(os.listdir(pdf_folder_path_b)))

#Testing DirectoryLoader with multithredding
#loader_cls=PyPDFium2Loader
#loader_kwargs=pdf_loader_kwargs
# pdf_loader_kwargs
loader = DirectoryLoader(root_dir,
                         glob="**/*.pdf",
                         use_multithreading=False,
                         show_progress=True,
                         loader_cls=PyPDFium2Loader)
docs = loader.load()

Number of pdfs from set 1:  37
Number of pdfs from set 2:  50


100%|██████████| 87/87 [02:22<00:00,  1.63s/it]


In [ ]:
#@markdown Creating metadata in dataframe format. [Completed!!]
#Extracting and adding the metadata from all docs
metadata_list = []
for i in range(len(docs)):
  # print(i)
  #Adding the filename
  docs[i].metadata['filename'] = docs[i].metadata['source'].split("/")[-1]
  #Adding the Set
  docs[i].metadata['set'] = docs[i].metadata['source'].split("/")[-2]
  #Adding the category for set 1
  if docs[i].metadata['set'] == 'Set1':
    docs[i].metadata['category'] = docs[i].metadata['filename'].split("_")[2]
  else:
    docs[i].metadata['category'] = 'NA'
  #Adding the charcter count
  docs[i].metadata['charcount'] = len(docs[i].page_content)
  #docs[i].metadata['content'] = docs[i].page_content
  metadata_list.append(docs[i].metadata)

#Creating a dataframe of all the metadata
metadata_df = pd.DataFrame(metadata_list)
print('Shape of metadata',metadata_df.shape)
metadata_df.head()


Shape of metadata (2547, 6)


,source,page,filename,set,category,charcount
0,/content/drive/MyDrive/Tiger-training/GenAI (T...,0,Black_Decker_AirCompresssor_ID_4700_Scanned.pdf,Set1,AirCompresssor,1
1,/content/drive/MyDrive/Tiger-training/GenAI (T...,1,Black_Decker_AirCompresssor_ID_4700_Scanned.pdf,Set1,AirCompresssor,1
2,/content/drive/MyDrive/Tiger-training/GenAI (T...,0,Black_Decker_AirCompresssor_ID_WX-RP810_Scanne...,Set1,AirCompresssor,57
3,/content/drive/MyDrive/Tiger-training/GenAI (T...,1,Black_Decker_AirCompresssor_ID_WX-RP810_Scanne...,Set1,AirCompresssor,57
4,/content/drive/MyDrive/Tiger-training/GenAI (T...,2,Black_Decker_AirCompresssor_ID_WX-RP810_Scanne...,Set1,AirCompresssor,57


In [ ]:

#@markdown  Choose your Text Splitter model
start = time.time()
splitter = "Langchain Recursive splitter" # @param ["\"Langchain Recursive splitter\""] {type:"raw", allow-input: true}
m = hashlib.md5()
if splitter == 'Langchain Recursive splitter':
  #Langchain Recursive splitter
  split_chunk_size = 350 # @param {type:"slider", min:100, max:500, step:5}
  split_overlap = 100 # @param {type:"slider", min:20, max:175, step:1}

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = split_chunk_size,
      chunk_overlap = split_overlap,
      length_function = len,
      separators=["\n\n", "\n", "\r\n","\t", "(?<=\. )"," ", ""]
  )



  documents_rcts = []

  for doc in tqdm(docs):
      url = doc.metadata['source']
      m.update(url.encode('utf-8'))
      uid = m.hexdigest()[:12]
      page = doc.metadata['page']+1
      filename = doc.metadata['filename']
      set_type = doc.metadata['set']
      category = doc.metadata['category']
      chunks = text_splitter.split_text(doc.page_content)
      for i, chunk in enumerate(chunks):
          documents_rcts.append({
              'id': f'{uid}-{i}',
              'text': chunk,
              'source': url,
              'page':page,
              'filename':filename,
              'set':set_type,
              'category':category
          })

  print(len(documents_rcts))
  texts_rcts = text_splitter.split_documents(docs)

end = time.time()
print('Time taken in Seconds:- ', end-start)



100%|██████████| 2547/2547 [00:00<00:00, 3185.02it/s]


42206
Time taken in Seconds:-  2.8569347858428955


Creating Embedding For text data & creating metadata in JSON

In [ ]:
#@markdown  Creating Metadata and Extracting text for Embeddings
start = time.time()
doc_rcts_df = pd.DataFrame(documents_rcts)
doc_rcts_df["chunk_length"] = doc_rcts_df["text"].apply(lambda x: len(x))
doc_rcts_df = doc_rcts_df.drop_duplicates(subset='text', keep="last")

df = doc_rcts_df[['id', 'filename', 'set', 'category', 'page']]

meta_data = []
for i in range(len(df)):
  d = {}
  for col in df.columns:
    d[col] = str(df.iloc[i][col])
  meta_data.append(d)
print('Numbers of Unique Chunks:-',len(meta_data))

end = time.time()
print('Time taken in Seconds:- ', end-start)

Numbers of Unique Chunks:- 32763
Time taken in Seconds:-  12.461097955703735


In [ ]:
texts_rcts[10]

Document(page_content='taking off parts, and before cleaning.\r\nAvoid contacting moving parts. ❍\t\r\nDo not operate any appliance with a damaged cord or ❍\t\r\nplug or after the appliance malfunctions, or is dropped or \r\ndamaged in any manner. Return appliance to the nearest \r\nauthorized service facility or call the appropriate toll-free', metadata={'source': '/content/drive/MyDrive/Tiger-training/GenAI (Training-2)/Case Study_Batch 2-20230928T185455Z-001/Case Study_Batch 2/Data/PDF Documents/Set1/Black_Decker_Blender_ID_bl2010bgc_Editable.pdf', 'page': 0, 'filename': 'Black_Decker_Blender_ID_bl2010bgc_Editable.pdf', 'set': 'Set1', 'category': 'Blender', 'charcount': 10326})

In [ ]:
doc_rcts_df.head()

,id,text,source,page,filename,set,category,chunk_length
7,f947a887ca67-0,Downloaded from www.Manualslib.com manuals sea...,/content/drive/MyDrive/Tiger-training/GenAI (T...,8,Black_Decker_AirCompresssor_ID_WX-RP810_Scanne...,Set1,AirCompresssor,56
8,c46f7e970475-0,"Size: 19"" x 19""\r\n10-SPEED BLENDER\r\nMÉLANGE...",/content/drive/MyDrive/Tiger-training/GenAI (T...,1,Black_Decker_Blender_ID_bl2010bgc_Editable.pdf,Set1,Blender,331
9,c46f7e970475-1,Read all instructions. ❍\t\r\nTo protect again...,/content/drive/MyDrive/Tiger-training/GenAI (T...,1,Black_Decker_Blender_ID_bl2010bgc_Editable.pdf,Set1,Blender,325
10,c46f7e970475-2,"taking off parts, and before cleaning.\r\nAvoi...",/content/drive/MyDrive/Tiger-training/GenAI (T...,1,Black_Decker_Blender_ID_bl2010bgc_Editable.pdf,Set1,Blender,310
11,c46f7e970475-3,authorized service facility or call the approp...,/content/drive/MyDrive/Tiger-training/GenAI (T...,1,Black_Decker_Blender_ID_bl2010bgc_Editable.pdf,Set1,Blender,321


In [ ]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


## Create the DB

This will take a bit of time on a T4 GPU

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

persist_directory = '/content/drive/MyDrive/GenAI (Training-2)/Case Study_Batch 2-20230928T185455Z-001/Case Study_Batch 2/Data'


Run Only if Necessary.

In [ ]:
## Here is the new embeddings being used

start = time.time()
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts_rcts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()
end = time.time()
print('Time taken by ChromaDB to load into Vector Database, along with Embeddings (in minutes):- ', (end-start)/60)

Time taken by ChromaDB to load into Vector Database, along with Embeddings (in minutes):-  93.18630072275798


In [ ]:
vectordb1 = Chroma(persist_directory=persist_directory, embedding_function=instructor_embeddings)
vectordb1.get

<bound method Chroma.get of <langchain_community.vectorstores.chroma.Chroma object at 0x7b736121f460>>

In [ ]:
retriever = vectordb1.as_retriever(search_kwargs={"k": 5})

Question & Answers Using Vector Database. Using Similarity Search for n = 3

In [ ]:
vectordb1.similarity_search('The purpose of the level command kit is to provide inflation control of your air helper springs. This kit will be an asset to your vehicle, meeting nearly all of your air supply needs.')

[Document(page_content='INSTALLATION INSTRUCTIONS\r\nCongratulations on your purchase of a new Level Command\r\nkit. This kit was designed to provide inflation control of your\r\nair helper springs. This kit will be an asset to your vehicle,\r\nmeeting nearly all of your air supply needs.\r\nPlease take a few minutes to read through the instructions,', metadata={'category': 'AirCompresssor', 'charcount': 2038, 'filename': 'Black_Decker_AirCompresssor_Nil1_Editable.pdf', 'page': 0, 'set': 'Set1', 'source': '/content/drive/MyDrive/GenAI (Training-2)/Case Study_Batch 2-20230928T185455Z-001/Case Study_Batch 2/Data/PDF Documents/Set1/Black_Decker_AirCompresssor_Nil1_Editable.pdf'}),
 Document(page_content='spring. Push the collar towards the body of the fitting and pull out the tube.\r\nSYSTEM OPERATION\r\nThe Level Command kit allows the air springs to be inflated from the inside of the vehicle. Push the paddle switch up to inflate\r\nthe air springs and push the paddle switch down to defl

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings,SentenceTransformerEmbeddings
#Vector Store
from langchain.vectorstores import Pinecone
import pinecone
# prompts
from langchain import PromptTemplate, LLMChain
# retrievers
from langchain.chains import RetrievalQA
#Importing time and textwrap
import textwrap
import time

#@markdown  Choose your model [Completed!!]
model_download = "databricks/dolly-v2-3b" # @param ["\"databricks/dolly-v2-3b\"", "\"meta-llama/Llama-2-7b\"", "\"google/flan-t5-large\"", "\"huggyllama/llama-7b\"", "None"] {type:"raw", allow-input: true}



#Models
def get_model(model):

    print('\nDownloading model: ', model, '\n\n')

    if model == 'llama-7b':
        model_repo = 'huggyllama/llama-7b'

        tokenizer = AutoTokenizer.from_pretrained(model_repo)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_8bit=True,
            # device_map='auto',
            torch_dtype=torch.float16,
            # low_cpu_mem_usage=True
        )

        max_len = 2048

    elif model == 'google/flan-t5-large':
        model_repo = 'google/flan-t5-large'

        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_4bit=True,
            # device_map='auto',
            torch_dtype=torch.float16,
            # low_cpu_mem_usage=True,
            trust_remote_code=True
        )

        max_len = 2048

    elif model == 'databricks/dolly-v2-3b':
        model_repo = 'databricks/dolly-v2-3b'

        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_4bit=True,
            # device_map='auto',
            torch_dtype=torch.float16,
            # low_cpu_mem_usage=True,
            trust_remote_code=True
        )

        max_len = 4096


    else:
        print("Not implemented model (tokenizer and backbone)")

    return tokenizer, model, max_len





In [ ]:
%%time
#model_download = "TheBloke/Llama-2-7B-Chat-GGML" #@param [ "facebook/opt-1.3b", "anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g", "anon8231489123/vicuna-13b-GPTQ-4bit-128g"] {allow-input: true}
#@markdown  Download the model [Completed!!]


tokenizer, model, max_len = get_model(model =model_download)

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

CPU times: user 11.5 s, sys: 23.9 s, total: 35.4 s
Wall time: 1min 51s


In [ ]:
temp=0
topp=0.95
reppenalty=1.15

#Hugging Face Pipeline
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    max_length = max_len,
    temperature = temp,
    top_p = topp,
    repetition_penalty = reppenalty
)

llm = HuggingFacePipeline(pipeline = pipe)

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
#Prompt Template
prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only pieces of context to answer the question at the end.
Use 100-150 words to anwer the question
{context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    # print('\n\nSources:')
    # for source in llm_response["source_documents"]:
    #     print(source.metadata['source'].split('/')[-1])
    return wrap_text_preserve_newlines(llm_response['result'])

Question Answering

In [ ]:

query = "'How to remove the sprocket cover?'"
start = time.time()
llm_response = qa_chain(query)
print('Query is:-', query, '\n')
print('LLM Answer is:- \n')
process_llm_response(llm_response)
end = time.time()
print('Time taken(Seconds): ', end-start)


Query is:- 'How to remove the sprocket cover?' 

LLM Answer is:- 

 The sprockets keep the chain moving, so removing a sprocket covers the chain from the motor.

1. Hold the sprocket cover down with your left hand.  With right hand, grab the sprocket cover with a pair of
needle nose pliers.  Push down on the sprocket cover until the spring loaded mechanism releases.  Pull the
sprocket cover out of the way.
2. Grab the sprocket cover with needle nose pliers and lift off the sprocket cover.
3. With right hand, grab the sprocket cover with needle nose pliers and pull away from the motor.
4. With right hand, grab the sprocket cover with needle nose pliers and twist clockwise.
5. Release the sprocket cover.


Time taken(Seconds):  13.243998527526855


In [ ]:

query = "'Are the charger and battery pack serviceable?'"
start = time.time()
llm_response = qa_chain(query)
print('Query is:-', query, '\n')
print('LLM Answer is:- \n')
process_llm_response(llm_response)
end = time.time()
print('Time taken(Seconds): ', end-start)


Query is:- 'Are the charger and battery pack serviceable?' 

LLM Answer is:- 

 No


Time taken(Seconds):  1.2956387996673584


In [ ]:
query = 'When should I avoid installing airline tubing?'
start = time.time()
llm_response = qa_chain(query)
print('Query is:-', query, '\n')
print('LLM Answer is:- \n')
process_llm_response(llm_response)
end = time.time()
print('Time taken(Seconds): ', end-start)


Query is:- When should I avoid installing airline tubing? 

LLM Answer is:- 

 Avoid using airline tubing if the following conditions apply:
1. The airline tubing is made of plastic (PVC is fine).
2. The airline tubing has been exposed to moisture such as rain or humidity.
3. The airline tubing is cut too short.
4. The airline tubing is folded or kinked.
5. The airline tubing is connected to fitting that is not properly tightened.


Time taken(Seconds):  13.79586672782898


In [ ]:
query = "'What to do if I already have an air suspension system?'"
start = time.time()
llm_response = qa_chain(query)
print('Query is:- \n', query, '\n')
print('LLM Answer is:- \n')
process_llm_response(llm_response)
end = time.time()
print('Time taken(Seconds): ', end-start)


Query is:- 
 'What to do if I already have an air suspension system?' 

LLM Answer is:- 

 You cannot replace or add parts to an existing air suspension system. Please refer to the installation
instructions for level command.

Installation Instructions


Time taken(Seconds):  3.2550928592681885


In [ ]:
query = "Can we use strong solvents on the compressor housing?"
start = time.time()
llm_response = qa_chain(query)
print('Query is:- \n', query, '\n')
print('LLM Answer is:- \n')
process_llm_response(llm_response)
end = time.time()
print('Time taken(Seconds): ', end-start)


Query is:- 
 Can we use strong solvents on the compressor housing? 

LLM Answer is:- 

 No, solvents can dissolve plastics and cause permanent damage.

A:

No, using solvents will permanently damage the plastic parts.


Time taken(Seconds):  4.368732213973999


Testing On Queries Given for Set1 and Set2

In [ ]:
xls = pd.ExcelFile('/content/Queries.xlsx')
df1 = pd.read_excel(xls, 'Queries_Set1')
df2 = pd.read_excel(xls, 'Queries_Set2')

In [ ]:
df1['Queries'][10]

'How to mount the compressor?'

picking random n Queries from set 1 and Generating their answers



In [ ]:
import random
n = 5
for i in random.sample(df1['Queries'].to_list(), n):
  print("Query is:- ", i)
  start = time.time()
  llm_response = qa_chain(i)
  process_llm_response(llm_response)
  end = time.time()
  print('Time taken: ', end-start, '\n')
  time.sleep(40)

Query is:-  Can I use this product for other than household use?
 No, you cannot use this product for other than household use.


Time taken:  2.438580274581909 

Query is:-  How to clean the motor filter? 
 To clean the motor filter, first remove the filter, then disassemble the motor, then clean the motor parts
using appropriate cleaning agent. Finally, reassemble the motor.


Time taken:  3.809100866317749 

Query is:-  What is the weight of Electric 18’’ Single Blade Lawn Mower Model 8000?
 The weight of this lawn mower model 8000 is 8.8 kg.

Electric single blade lawn mowers have become very popular over the past few years due to their simplicity,
low cost, and ability to cut grasses efficiently.

However, they also come with a heavyweight. The weight of this electric lawn mower is 8.8 kilograms.


Time taken:  7.214718341827393 

Query is:-  How to lubricate the mower?
 Lubricate the mower by applying a light film of lubrication to the exposed blade.  Insert the battery and
turn 

picking random n Queries from set 2 and Generating their answers



In [ ]:
import random
n = 5
for i in random.sample(df2['Queries'].to_list(), n):
  print("Query is:- ", i)
  start = time.time()
  llm_response = qa_chain(i)
  process_llm_response(llm_response)
  end = time.time()
  print('Time taken: ', end-start, '\n')
  time.sleep(40)

Query is:-  How to turn on the 20V Max* Brushless Cordless Compact 1/2" (13 mm) Drill/Driver and 20V Max* Brushless Cordless Compact 1/2" (13 mm) Drill/Driver/Hammerdrill?
 You can use your thumb to depress the power button for 10 seconds.


Time taken:  2.6816580295562744 

Query is:-  How to charge the battery of 60V Max* Lithium Handheld Blower?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 The charger is designed to use standard household 120 volt 60Hz power. First, plug the charger into any
standard 120 volts 60 Hz electrical outlet. Second, slide the charger onto the battery pack as shown in Figure
1 and let it charge initially for the lifetime of the battery, it is recommended to keep the battery connected
to the charger whenever the tool is not in use. When the battery is full, the power consumption of the charger
is negligible. Do not charge the battery at ambient temperatures below 5°C or above 40°C. Recommended charging
temperature: approximately 24°C.


Time taken:  10.389633417129517 

Query is:-  How to shut down the 135 PSI Max. 3 Gallon Air Compressor?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



  1. Move the on /off switch to the Off Position
    2. Turn the regulator knob clockwise until it is fully closed
      and ensure the gauge reads 0 PSI
    3. Drain the air tanks
    4. Allow the compressor to cool down


Time taken:  6.3120787143707275 

Query is:-  What is the use of dual range gearing?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Dual range gearing allows you to shift gears for greater versatility.


Time taken:  1.9550161361694336 

Query is:-  How to stop the 20V Max* Axial Blower?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 1. Press the reset button until it stops 2. Wait 10 seconds 3. Remove battery 4. Disconnect the blower tube.


Time taken:  3.229461908340454 



Evaluation of LLM Responses (WIP)

Issues Facing with Running LLM:
1. Out of GPU
2. Creating Vector embedding everytime to run LLM (60-70 minutes)
3. Can't Run All the query in a loop (OutOfMemoryError: CUDA out of memory.)    

Testing with labelled Data

In [ ]:
!pip install evaluate
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8a37462f609ecfc81e2a2a60357de99b67c10806b65bc843e991796cba0f0506
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
# import evaluate

from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [ ]:
path = '/content/drive/MyDrive/GenAI (Training-2)/Case Study_Batch 2-20230928T185455Z-001/Case Study_Batch 2/Data/Labelled Data/Labelled_Data_Set1.xlsx'
def evaluate(query):
    ground_truth = pd.read_excel(path)
    ground_truth = ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0]
    if len(ground_truth) == 0:
      return ('Query not found in Labelled Data')

    print('Query is:- \n', query, '\n')
    print('Labelled Answer for above Query is:- \n')
    for i in ground_truth.split('.'):
      print(i, end = '.\n')
    llm_response = qa_chain(query)
    print('\n')
    print(' Answer from LLM is:- \n')
    predictions = process_llm_response(llm_response)

    # bleu = evaluate.load("bleu")
    # rouge = evaluate.load('rouge')
    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    # bleu_score = bleu.compute(predictions=predictions, references=ground_truth)
    bleu_score = sentence_bleu(ground_truth, predictions)
    rouge_score = rouge.score(ground_truth, predictions)['rougeL'].fmeasure
    print('\n')
    return bleu_score, rouge_score, predictions



In [ ]:
ground_truth = pd.read_excel(path)
ground_truth['Queries'].to_list()

['What is the use of level command kit?',
 'What to do if I already have an air suspension system?',
 'When should I avoid installing airline tubing?',
 'How can I connect the airline tubing?',
 'Give the list of parts available in the level command kit.',
 'Give the list of parts available in the level command kit with the number of items available in it.',
 'Give me the instructions to install level command.',
 'What are the tools required to install?',
 nan,
 nan,
 'What to do after selecting a mounting location for the control panel?',
 'How can I prepare the compressor to mount?',
 'How to mount the compressor?',
 'Is there any precaution that needs to be taken before installing the airline tubing?',
 'What precaution should I take before installing the airline tubing?',
 'How to release the air pressure from the air springs?',
 'How to attach the control panel to the dashboard?',
 'What to do after attaching the control panel to the dashboard?',
 'Where should we connect the red 

In [ ]:
query =   'Define the signal to noise ratio of WX-RP810'
ans = ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0]
ans

'More than 60 dB (60 dBµV Input, 15 kHz FM A-weighted)'

In [ ]:
questions = []
label_answer = []
llm_answer = []
b_score = []
r_score = []


In [ ]:
query =   'Define the signal to noise ratio of WX-RP810'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)

Query is:- 
 Define the signal to noise ratio of WX-RP810 

Labelled Answer for above Query is:- 

More than 60 dB (60 dBµV Input, 15 kHz FM A-weighted).


 Answer from LLM is:- 

 The signal to noise ratio of a radio receiver can be calculated as S / N where S is the strength of the
received signal and N is the noise floor. In this case, S would be the measured power of the received signal
and N would be 0 dBA. The signal to noise ratio is therefore 2 dBA - 3.0 dBA =  1.9 dBA which is below the
limit of 3 dBA set by the government of Canada.




BLEU Score : 8.892271542172293e-232
ROUGE Score : 0.02197802197802198


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query =   'What are the accessories required for air compressor?'


bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)

Query is:- 
 What are the accessories required for air compressor? 

Labelled Answer for above Query is:- 

1.
 No.
 4750 1/10 - Gallon Sealant/Adhesive Dispenser
 2.
 No.
 4751 1- Quart Adhesive Dispenser
 3.
 No.
 37536 - Extension Hose, 15 ft.
,
 4.
 No.
 37537 - Extension Hose, 25 ft.
,
 5.
 No.
 37535 - 2 Coupling Adaptors
 6.
 No.
 U2106 - Safety Glasses
 7.
 NO.
 37525 - QUICK-DISCONNECT SET.


 Answer from LLM is:- 

 A 4-way socket, 10-gauge stranded cable,  air filter, oil filter, oil, and a 12" gauge 2-in rigid steel pipe.




BLEU Score : 1.2522435979686076e-231
ROUGE Score : 0.05797101449275362


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query =   'Why should I avoid oiling the compressor?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)

Query is:- 
 Why should I avoid oiling the compressor? 

Labelled Answer for above Query is:- 

You'll never need to oil the compressor and will always get clean, dry compressed air with no atomized oil to contaminate a spray painted surface.
.


 Answer from LLM is:- 

 Oil can attract dirt and other matter which can get into the moving parts of the compressor and cause damage
over time.  Oiling a compressor could lead to serious injury or death if someone were to come into contact
with the moving parts.

Avoid using oils, lubricants, or solvents near electrical equipment.
Never operate the compressor outdoors when it's raining or in wet conditions.
Never operate the compressor with protective covers removed or damaged.




BLEU Score : 8.702337104256174e-232
ROUGE Score : 0.1188118811881188


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query =   'What to do after selecting a mounting location for the control panel?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)

Query is:- 
 What to do after selecting a mounting location for the control panel? 

Labelled Answer for above Query is:- 

After selecting a mounting location for the control panel prepare the compressor.
 Install the rubber isolator feet to the compressor from the bottom of the compressor foot.
 Insert the brass sleeve into the top of the rubber isolator foot.
 Install the push-to-connect male fitting into the threaded exhaust port on the compressor head.
 Tighten the fitting sufficiently to engage at least two threads with pre-applied orange thread sealant.
 DO NOT OVERTIGHTEN THE FITTING.
 Install the air filter into the threaded inlet port on the compressor head (finger tight).
.


 Answer from LLM is:- 

 1.  Pull the anti-kickback assembly up so that the locking tab is flush against the side of the assembly. 2.
Push down on the anti-kickback assembly until the locking tab clicks into place.




BLEU Score : 1.0599580721425215e-231
ROUGE Score : 0.15873015873015872


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query =   'How to switch ON and switch OFF the compressor?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 How to switch ON and switch OFF the compressor? 

Labelled Answer for above Query is:- 

To turn the compressor "ON," push down on the side of the rocker switch marked "ON.
" Pushing down on the other side of switch turns the compressor "OFF.
" Be sure to push the switch "OFF" before disconnecting the plug.
 Otherwise the compressor will start imme diately the next time it is plugged in.
.


 Answer from LLM is:- 


1. To switch ON the compressor, move the ON/OFF switch to the ON position.
The tank must be full for this step.
2. To switch OFF the compressor, move the ON/OFF switch to the OFF position.
NOTE: Do not remove the tank from the compressor. Doing so may damage the tank.
3. After using the compressor, close the regulator knob by turning it clockwise.
NOTE: Do not over-tighten the regulator knob as doing so can damage the unit.
4. Disconnect the power cord from the compressor.




BLEU Score : 8.953390407092663e-232
ROUGE Score : 0.2676056338028169


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query =   'How to clean the air compressor?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 How to clean the air compressor? 

Labelled Answer for above Query is:- 

A cloth dampened with mineral spirits or an ammonia solution can be used to soften and remove adhesives, caulking compound, or dirt from the compressor housing.
.


 Answer from LLM is:- 

 1.Turn off the compressor. 2.Remove air compressor plug from outlet. 3.Drain air tank. 4.Allow air compressor
to cool down before starting service.




BLEU Score : 1.1522277201606316e-231
ROUGE Score : 0.11538461538461539


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query =   'How to release the air pressure from the air springs?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 How to release the air pressure from the air springs? 

Labelled Answer for above Query is:- 

To release the air pressure, remove the valve core from the manual inflation valves or release the pressure by using a tire gauge to depress the valve stem.
.


 Answer from LLM is:- 

 Remove the valve core from the manual inflation valves or release the pressure by using a tire gauge to
depress the valve stem.





BLEU Score : 1.1662621194838564e-231
ROUGE Score : 0.9019607843137255


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query =  'How to use the adaptors available in the quick disconnect set?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 How to use the adaptors available in the quick disconnect set? 

Labelled Answer for above Query is:- 

The Adaptors can be attached to your compressor accessories and the Fitting to your compressor hose.
 Connection or removal can be made instantly without twisting the hose or accessory.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 


  1. Turn on the power of the electric drill.
2. Hold the black sleeve of the quick release chuck with your left hand and hold the base of the drill with
your right hand.
3. Push the red button on the top of the quick release chuck until it clicks.
4. Release the black sleeve of the quick release chuck.
5. Grab the orange handle of the quick release chuck with your right hand and lift the orange handle up.
6. Pull the orange handle down until the orange handle stops moving.
7. Lift the orange handle again and the orange handle will retract into the body of the quick release chuck.
8. Now you can put the new accessory into the quick release chuck.




BLEU Score : 7.88041229957172e-232
ROUGE Score : 0.1038961038961039


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query = 'What to do to prevent loss of power and overheating?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)


Query is:- 
 What to do to prevent loss of power and overheating? 

Labelled Answer for above Query is:- 

To prevent loss of power and overheating, ensure that the wires in an extension cord are of adequate size.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


RuntimeError: ignored

In [ ]:
query = 'When should I clean the air compressor?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 When should I clean the air compressor? 

Labelled Answer for above Query is:- 

The air compressor should be cleaned immediately after use to help ensure its long life and attractive appearance.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 

 You should clean the air compressor every three months. This will help prevent buildup of dust and other
particles inside the system. Cleaning the system will also ensure it is running at peak efficiency.

A:

You can clean your air compressor by draining the air tank.  This will remove any accumulated fluid and allow
for better cooling.




BLEU Score : 9.27755966550629e-232
ROUGE Score : 0.16


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query = 'Can I work from a greater distance while using Black & Decker hose extensions?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 Can I work from a greater distance while using Black & Decker hose extensions? 

Labelled Answer for above Query is:- 

With Black & Decker hose extensions, you can work at a greater distance from the compressor.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 

 Yes, but it's important to keep in mind that longer lengths of hose may become less effective over time due
to friction caused by movement and heat from the motor. It is best to keep them near the unit where they will
get the most use.




BLEU Score : 1.0157715464244269e-231
ROUGE Score : 0.0967741935483871


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query = 'Can we use duplicate parts while repairing damaged cords?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 Can we use duplicate parts while repairing damaged cords? 

Labelled Answer for above Query is:- 

No, when servicing double-insulated tools, it is important to use only identical replacement parts.
 This ensures the continued integrity of the double insulation system and maintains safety standards.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 

 No, as unauthorized repairs could cause damage to the appliance.




BLEU Score : 1.3466263920541852e-231
ROUGE Score : 0.15384615384615385


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query = 'How far can I work when using Black & Decker hose extensions?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 How far can I work when using Black & Decker hose extensions? 

Labelled Answer for above Query is:- 

You can go as far as 200 ft.
 away from the compressor and still be assured of adequate air flow and pressure at the point of use.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 

 You cannot work on your tools if they are plugged into a power source. Always unplug your tools before moving
them.

The maximum distance you can work while using these extensions is 12.7 millimeters (0.5 inch).




BLEU Score : 1.0650987681285846e-231
ROUGE Score : 0.09230769230769231


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query = 'Is grounding required for tools built with this insulation system?'
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 Is grounding required for tools built with this insulation system? 

Labelled Answer for above Query is:- 

No, tools built with the double insulation system are not intended to be grounded.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 

 No, grounding is not required for tools built with this insulation system.

A:

No, grounding is not required for tools built with this insulation system.




BLEU Score : 1.101763128289178e-231
ROUGE Score : 0.358974358974359


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query = "How to mount the compressor?"
bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 How to mount the compressor? 

Labelled Answer for above Query is:- 

To mount the compressor begin the process by removing the negative battery cable.
 Select a convenient location to mount the compressor.
 This location should provide ample air flow and be protected from airborne debris and moisture.
 The mounting surface should be rigid to support the compressor, such as under the hood on a fender well or in a vented storage compartment.
 The compressor is oil-less and can be mounted in any orientation necessary for installation.
 Using the compressor template and a center punch, mark and drill three 3/16" holes.
 Any burrs in the holes should be removed to prevent damage to the rubber isolators.
 Mount the compressor using the supplied 10 -32 x 1" machine screws, 10 -32 lock nuts, and 3/16" washers see Figure "C".
 Maximum
 vibration isolation can be achieved by properly mounting the compressor.
 The
 machine screw and nut should be tightened only enough to bottom-out 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 

 Use the provided mounting hardware to mount the compressor.  Mark the mounting locations on the compressor
body with a center punch then drill corresponding holes. Tighten the screws until they bottom out the brass
inserts.  The vibration isolation isolators are flexible so do not overtighten them.  Once the isolators are
installed, install the control panel and test it.  If there is too much noise, remove one isolator and replace
it.




/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_



BLEU Score : 9.455536398767114e-232
ROUGE Score : 0.2231404958677686


In [ ]:
query = "What type of extension cord should be used when operating the compressor?"

bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 What type of extension cord should be used when operating the compressor? 

Labelled Answer for above Query is:- 

When operating the compressor outdoors, you should use only U.
L.
 listed "OUTDOOR" extension cords.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 

 A cord with an AWG of 16 or higher is recommended.  An AWG of 12-14 is considered acceptable but may result
in reduced performance due to increased resistance from the insulation.




BLEU Score : 1.0489335475684504e-231
ROUGE Score : 0.0425531914893617


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
query = "What does the guarantee cover in terms of defects?"

bleu_score, rouge_score, prediction = evaluate(query)
print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)
label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
questions.append(query)
llm_answer.append(prediction)
b_score.append(bleu_score)
r_score.append(rouge_score)
time.sleep(200)

Query is:- 
 What does the guarantee cover in terms of defects? 

Labelled Answer for above Query is:- 

The guarantee covers the correction, by repair or parts replacement at no charge, of any defect arising from faulty material or workmanship.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




 Answer from LLM is:- 

 The guarantee covers any defect in material or workmanship provided, but Applica's liability will not exceed
the purchase price of the product. For how long? Two years from the date of original purchase.




BLEU Score : 1.0858875540217369e-231
ROUGE Score : 0.2857142857142857


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:

for i in ground_truth['Queries'].to_list()[20:]:
  query = i

  bleu_score, rouge_score, prediction = evaluate(query)
  print('BLEU Score :', bleu_score)
  print('ROUGE Score :', rouge_score)
  label_answer.append(ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0])
  questions.append(query)
  llm_answer.append(prediction)
  b_score.append(bleu_score)
  r_score.append(rouge_score)
  time.sleep(200)

Query is:- 
 How can I check my system with installed level command kit? 

Labelled Answer for above Query is:- 

With the Level Command kit and your air helper springs installed, you are ready to test the system.
 Reattach the negative battery cable.
 Turn on the vehicle's ignition.
 Push the paddle switch up to inflate the air springs.
 The gauge will display how much air pressure is in the air springs.
 Inflate the air helper springs to 70 psi and check the fittings for air leaks with an applied solution of soap and water.
 If a leak is detected at a tubing connection, check to make sure that the tube is cut as square as possible and that it is pushed completely into the fitting.
 The tubing can easily be removed from the fitting.
 First, release the pressure from the air spring.
 Push the collar towards the body of the fitting and pull out the tube.
.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
dict = {'Question': questions, 'Labbeled Answer': label_answer, 'LLM Response': llm_answer, 'Blue Score': b_score, 'Rouge Score': r_score}

df = pd.DataFrame(dict)

In [ ]:
df.drop_duplicates()

,Question,Labbeled Answer,LLM Response,Blue Score,Rouge Score
0,Define the signal to noise ratio of WX-RP810,"More than 60 dB (60 dBµV Input, 15 kHz FM A-we...",The signal to noise ratio of a radio receiver...,8.892272e-232,0.021978
1,What are the accessories required for air comp...,1. No. 4750 1/10 - Gallon Sealant/Adhesive Dis...,"A 4-way socket, 10-gauge stranded cable, air...",1.252244e-231,0.057971
2,What to do after selecting a mounting location...,After selecting a mounting location for the co...,1. Pull the anti-kickback assembly up so tha...,1.059958e-231,0.158730
3,Why should I avoid oiling the compressor?,You'll never need to oil the compressor and wi...,Oil can attract dirt and other matter which c...,8.702337e-232,0.118812
4,How to switch ON and switch OFF the compressor?,"To turn the compressor ""ON,"" push down on the ...","\n1. To switch ON the compressor, move the ON/...",8.953390e-232,0.267606
5,How to clean the air compressor?,A cloth dampened with mineral spirits or an am...,1.Turn off the compressor. 2.Remove air compr...,1.152228e-231,0.115385
6,How to release the air pressure from the air s...,"To release the air pressure, remove the valve ...",Remove the valve core from the manual inflati...,1.166262e-231,0.901961
7,How to use the adaptors available in the quick...,The Adaptors can be attached to your compresso...,\n 1. Turn on the power of the electric drill...,7.880412e-232,0.103896
8,When should I clean the air compressor?,The air compressor should be cleaned immediate...,You should clean the air compressor every thr...,9.277560e-232,0.160000
9,Can I work from a greater distance while using...,"With Black & Decker hose extensions, you can w...","Yes, but it's important to keep in mind that ...",1.015772e-231,0.096774


In [ ]:
df.to_csv('llm_report.csv')